# Import Statements

In [1]:
from numpy.random import seed
seed(888)
import tensorflow as tf
tf.random.set_seed(404)

In [2]:
import os
import numpy as np
import tensorflow as tf

# Constants

In [8]:
X_TRAIN_PATH = 'MNIST/digit_xtrain.csv'
X_TEST_PATH = 'MNIST/digit_xtest.csv'
Y_TRAIN_PATH = 'MNIST/digit_ytrain.csv'
Y_TEST_PATH = 'MNIST/digit_ytest.csv'

# Gather Data

In [5]:
%%time
y_train_all = np.loadtxt(Y_TRAIN_PATH, delimiter=',', dtype=int)

CPU times: total: 78.1 ms
Wall time: 99.1 ms


In [6]:
y_train_all.shape

(60000,)

In [9]:
y_test = np.loadtxt(Y_TEST_PATH, delimiter=',', dtype=int)

In [10]:
%%time
x_train_all = np.loadtxt(X_TRAIN_PATH, delimiter=',', dtype=int)


CPU times: total: 34 s
Wall time: 40 s


In [11]:
%%time
x_test = np.loadtxt(X_TEST_PATH, delimiter=',', dtype=int)

CPU times: total: 5.94 s
Wall time: 6.98 s


# Data Exploration

In [12]:
x_train_all.shape
# 784 => 28(width) * 28(height) * 1(nr. of channels)

(60000, 784)

In [14]:
x_train_all[0]
# 0 => very white, 255 => very black,in-between => shades of gray

array([  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   3,  18,  18,  18,
       126, 136, 175,  26, 166, 255, 247, 127,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,  30,  36,  94, 154, 17

In [15]:
x_test.shape

(10000, 784)

In [17]:
y_train_all.shape

(60000,)

In [23]:
y_train_all[:5]

array([5, 0, 4, 1, 9])